In [5]:
!pip install langchain langchain-community transformers sentence-transformers

  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.75-py3-none-any.whl.metadata (5.7 kB)
  Using cached langchain_text_splitters-0.3.10-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.4.20-py3-none-any.whl.metadata (14 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sqlalchemy-2.0.43-cp312-cp312-win_amd64.whl.metadata (9.8 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
  Using cached ty

ERROR: To modify pip, please run the following command:
C:\Users\eourm\anaconda3\envs\LLM\python.exe -m pip install langchain langchain-community transformers sentence-transformers


In [19]:
# Step 1) Prompt → Model (LLM)
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate, ChatPromptTemplate

generator = pipeline("text2text-generation",
                     model="gogamza/kobart-base-v2",
                     max_new_tokens=128)
llm = HuggingFacePipeline(pipeline=generator)

# 영어 지시가 더 잘 먹힙니다
prompt = PromptTemplate.from_template("한국어로 한 줄 요약: {text}")
chain = prompt | llm

out = chain.invoke({"text": "LangChain helps you build LLM apps easily."})
print(">>", repr(out))   # repr로 빈 문자열 여부 확인

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
Device set to use cpu


>> '한국 한 한 줄 요약: LangChain helps you build LLM apps easild LLM apps easily.. build LLM apps easps easily.. 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한국 한 줄 요약: LangChain helps you build LLM apps you buil'


In [20]:
# Step 2) 검색(Retriever) → Prompt → Model
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline


texts = [
"LangChain은 LLM 앱 개발을 돕는 프레임워크다.",
"RAG는 검색된 컨텍스트를 활용해 더 정확히 답한다.",
"나는 배고프다.",
"졸려 죽겠다."
]
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_texts(texts, emb)


question = "RAG는 무엇인가요?"
context = "\n".join([d.page_content for d in db.similarity_search(question, k=2)])



generator = pipeline("text2text-generation",
                     model="gogamza/kobart-base-v2",
                     max_new_tokens=128)
llm = HuggingFacePipeline(pipeline=generator)
prompt = PromptTemplate.from_template("""
아래 컨텍스트만 근거로 질문에 한국어로 간결히 답하세요.
[컨텍스트]
{ctx}
---
[질문] {q}
[답변]
""")
rag_chain = prompt | llm
print(rag_chain.invoke({"ctx": context, "q": question}))

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
Device set to use cpu


아래 컨텍스트만 근거로 질문에 한국어로 한국어로 간결히 답하세요.
[컨텍스트]
나는 배고프다.
RAG는 검색된 컨텍스트를 활용해 더 정확히 답한다.
--- . 잘 잘 잘한다.
--- 잘 잘 답한다.
 잘 정확히 답한다.
--- 잘 잘 잘한다.
--- 잘 잘 잘 잘 잘 잘 잘 잘 잘한다.
 잘 잘 잘 잘 잘 잘 잘 잘 잘 잘 잘 잘 한국어로 한국어로 한국어로 간결히 답하세요.
[럼아아아래 컨텍스트만 근거로 질문에 한국어로 한국어로 한국어로 간결히 답하세요.
[


In [22]:
# Step 3) LCEL (LangChain Expression Language)로 전체 파이프 구성
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableLambda
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline


# 1) 데이터 & 검색기
texts = ["LangChain은 체인 기반 LLM 앱 프레임워크.", "FAISS는 빠른 벡터 검색 라이브러리.", 
         "RAG는 검색 기반 생성.", "나는 배고프다.", "졸려 죽겠다.", "커피 한 잔 해야지."]
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_texts(texts, emb)
retriever = RunnableLambda(lambda q: db.similarity_search(q, k=2))


# 2) 문서 포맷터
fmt = RunnableLambda(lambda docs: "\n".join(d.page_content for d in docs))


# 3) 프롬프트 & 모델
prompt = PromptTemplate.from_template("""
컨텍스트만 이용해 질문에 답하세요.
[컨텍스트]
{context}
---
[질문] {question}
[답변]
""")

generator = pipeline("text2text-generation",
                     model="gogamza/kobart-base-v2",
                     do_sample=False,
                     max_new_tokens=128)
llm = HuggingFacePipeline(pipeline=generator)


# 4) 전체 파이프: 질문(str) → 검색 → 텍스트화 → 프롬프트 바인딩 → LLM
chain = (RunnableLambda(lambda x: x)
| {"context": retriever | fmt, "question": RunnableLambda(lambda x: x)}
| prompt | llm)


print(chain.invoke("LangChain은 무엇인가요?"))

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
Device set to use cpu


스트만 이용해 질문에 답하세요.
[컨텍스트]
LangChain은 체인 기반 LLM 앱 프레임워크.
커피 한 잔 해야지.
---
[질문] LLM 앱 프레임워크.
커피 한 잔 해야지.
---
[질문] LangChain은 무엇인가요?
[답변]
[답변]
[답변]
[답변]
[답변]
[컨텍스트만 이용해 질문에 답하세요.
[컨텍스트만 이용해 질문에 답하세요.
[컨텍스트만 이용해 질문에 답하세요.
[컨텍스트만 이용해 질문에 답
